In [1]:
%run imports_and_functions
%run sep_id_functions

downsample_ratio=1

In [2]:
params = InputParameters(
    window_size = timedelta(days=728), #approx 2 years
    step_size = timedelta(days=546), #approx 18 months (three quarters of the window_size)
    first_year=1982, last_year=2022,
    min_intensity=0.2, min_duration=5,
    min_pixels = int(250_000/(downsample_ratio**2)),
    output_folder = '/media/eoserver/AnaO_ATL/JP_ATL/mhw_labeled_dataset/2d_detection/',
    mhw_output_folder =  '/media/eoserver/AnaO_ATL/JP_ATL/mhw_labeled_dataset/2d_detection/detected_mhws/CHECK!!!',
    reset=True)

In [3]:
lbl_dates_dir = '/home/eoserver/beatriz/JP/data/lbl_dates_no_overlap.pkl'
with open(lbl_dates_dir, 'rb') as f:
    lbl_dates = pickle.load(f)
    
labels = lbl_dates.keys()

folder_path = params.getp('output_folder')  # replace with your folder path
substring = '.'
files_with_substring = get_files_with_substring(folder_path, substring)
files = [folder_path + file for file in files_with_substring]

dataset = xr.open_mfdataset(files, chunks={'time': 200})
time_arr = dataset.time.values
latitudes = dataset['lat'].values
longitudes = dataset['lon'].values

In [4]:
NAO_df = pd.read_csv('/home/eoserver/beatriz/JP/data/NAO_data_update.csv')
NAO_df = NAO_df.rename(columns={NAO_df.columns[0]: 'Year'})

prov_arrays = get_province_asarray(map_bounds = (len(dataset.lat.values), len(dataset.lon.values)))

In [ ]:
serial_number = 1
for lbl in list(labels):
    start_date = to_datetime(lbl_dates[lbl][0])
    for end_date in lbl_dates[lbl][1:]:
        end_date = to_datetime(end_date)
        if (end_date-start_date).days >=params.getp('min_duration'):
            str1 = start_date.strftime('%Y-%m-%d'); str2 = end_date.strftime('%Y-%m-%d')
            
            chunk = dataset.sel(time=slice(start_date, end_date))
            time_range = chunk.time.values
            
            frames = chunk['mhw_label'].values; del chunk
            frames = np.where(frames!=lbl, 0, 1)
            
            total_pixels = np.count_nonzero(frames)
            if total_pixels >= params.getp('min_pixels'):
                print(f'mhw {serial_number} from label {lbl}: from {str1} to {str2}; pixels: {total_pixels}')
                
                mhw_data = get_mhw_ids(frames, start_date, end_date, prov_arrays, NAO_df, serial_number, total_pixels)
                serial_number += 1
                
                mhw_data['time_array'] = time_range
                save_output_nc(mhw_data, params, latitudes, longitudes)
                
                """ name = mhw_data['ID']
                with open(params.getp('mhw_output_folder') + name + '.pkl', 'wb') as f:
                    pickle.dump(mhw_data, f) """
                    
                del mhw_data
            
            del frames
        start_date = end_date + timedelta(days=1)

dataset.close()

CHECK RESULTS

In [ ]:
def extract_year(s):
    last_part = s.rsplit('/', 1)[-1]
    parts = last_part.split('_')
    year = int(parts[2])
    return year

folder_path = params.getp('mhw_output_folder')
substring = '.'
files_with_substring = get_files_with_substring(folder_path, substring)
files = [folder_path + file for file in files_with_substring]
files = sorted(files, key=extract_year)
files

In [12]:
for i, file in enumerate(files):
    with open(file, 'rb') as f:
        mhw = pickle.load(f)

    filename = file.rsplit('/', 1)[-1]
    save_video(mhw['Areas'], f'./videos/{i}_{filename[:-4]}.mp4', labeling= False)